In [37]:
class HeatMap(object):
   
    
    def __init__(self, folder):
        self.folder = folder
        self.freqThreshold = 300
        self.deltaZ = 3
        self.initial = 0
        self.fileOut = self.folder + "heatMap.jpg"
        
    def set_subsystem(self,subSys):
        self.subsystem = subSys
        
    def set_fileOut(self, fileOut):
        self.fileOut = self.folder + fileOut
        
    def set_system(self,*args):
        self.system = []
        for system in args:
            self.system.append(system)
            
    def set_numbers(self, *args):
        self.number = []
        for number in args:
            self.number.append(number)
    def set_inital(self, initial):
        self.initial = initial
        
    def set_deltaZ(self, deltaZ):
        self.deltaZ = deltaZ
        
    def set_freqThreshold(self, freqThreshold):
        self.freqThreshold = freqThreshold
        
    def getZVal(self, end,start):
        for index in range(start,end):
            yield self.initial + index * self.deltaZ
            
    def genFiles( self, number, system ):
        for index in range(1,number + 1):
            yield system +"_%d_%s.pspect"%(index, self.subsystem)
    
    def getDataFrame(self, list_names):
        import pandas as pd
        freqThreshold = self.freqThreshold
        folder = self.folder
        for file in list_names:
            df1 = pd.read_csv(folder+file,delimiter='\t',names=["f","I","smooth"])
            df1.I = df1.I/sum(df1.I)
            df1.smooth = df1.smooth/sum(df1.smooth)
            df1.fillna(0, inplace = True)
            df1 = df1[df1.f<freqThreshold]
            yield df1
            
    def getHeatMap(self, dfList):
        import pandas as pd
        columns = len(dfList)
        rows = len(dfList[0].iloc[:,0])
    
        from numpy import zeros
        heatMapMat = zeros(rows*columns).reshape(rows,columns)
        positionMat = [(x,y) for x in range(rows) for y in range(columns)]
        for matPos in positionMat:
            heatMapMat[matPos[0],matPos[1]] = dfList[matPos[1]].smooth[matPos[0]]
        return heatMapMat
    
    def HeatMapMatrix(self):
        self.mat = []
        for index in range(len(self.system)):
            fileList=list(self.genFiles(self.number[index],self.system[index]))
            dfList = list(self.getDataFrame(fileList))
            mat.append(self.getHeatMap(dfList))
            
    
        
    
    
    def heatMapPlot(self):
   
        import numpy as np
        import matplotlib.cm as cm
        import matplotlib.pyplot as plt
        import matplotlib.cbook as cbook
        from matplotlib.path import Path
        from matplotlib.patches import PathPatch
    
    #---------------------------------------
    #subplot adjust
        left  =0.1# the left side of the subplots of the figure
        right = 0.9    # the right side of the subplots of the figure
        bottom = 0.1   # the bottom of the subplots of the figure
        top = 0.9      # the top of the subplots of the figure
        wspace = 0.2  # the amount of width reserved for blank space between subplots,
               # expressed as a fraction of the average axis width
        hspace = 0.2  # the amount of height reserved for white space between subplots,
               # expressed as a fraction of the average axis height
        fig, (ax1, ax2) = plt.subplots(1, 2, sharey= True)
    #fig.subplots_adjust(left=left,right=right,bottom=bottom,top=top\
    #                  ,wspace=wspace,hspace=hspace)
    
        n1 = self.number[0]
        n2 = self.number[1]
        freqThreshold = self.freqThreshold
        deltax = int(900/sum(self.number))
        deltay = int(freqThreshold/20)
    
        
        fig.set_size_inches(6,6)
        zVal1=list(getZVal(n1, 0))
        zVal2=list(getZVal(n2,n1))
        axis1= n1 * deltax
        axis2= n2 * deltax
        mat1 = self.mat[0]
        mat2 = self.mat[1]
        
        
        im1 = ax1.imshow(mat1,interpolation='bilinear',extent=[0, axis1, 0, 1000],
               vmax=mat2.max(), vmin=mat2.min())
        ax1.set_xticks(range(0,axis1,deltax))
        ax1.set_xticklabels(list(range(0,n1*3,3)))
        ax1.set_yticks(range(0,1000,50))
        ax1.set_yticklabels(range(freqThreshold,0,-deltay))


        im2 = ax2.imshow(mat2,interpolation='bilinear',extent=[0, axis2, 0, 1000],
               vmax=mat2.max(), vmin=mat2.min())
        ax2.set_xticks(range(0,axis2,2*deltax))
        ax2.set_xticklabels(range(18,18+n2*3,3*2))
        ax2.set_aspect('auto')
        colorbar_ax=fig.add_axes([0.95,0.1,0.05,0.8])
        fig.colorbar(im2,cax=colorbar_ax)
        plt.subplots_adjust(wspace = 0, hspace = 0)
        plt.title(fileout.split("/")[-1].split(".")[0])
        plt.savefig(fileout, bbox_inches="tight")
        plt.show()
    
    
        
    
    
    

In [42]:
def getAllPlots(subsystem):
    folder = "/afs/crc.nd.edu/user/h/hbhattar/Hemanta/metals/ThermalConductivity/EqulibratedSystem/"
    fileOut = "%s.eps"%(subsystem)
    tailSubsystem = "Vcorr_smooth" 
    system2 = "%s_Dynamics_TimeAdjusted_CM_Atomic"%(subsystem)
    system1 = "%s_Dynamics_TimeAdjusted_Au_Atomic"%(subsystem)
    n1=6
    n2=19
    heatMap = HeatMap(folder)
    heatMap.set_numbers(n1,n2)
    heatMap.set_system(system1, system2)
    heatMap.set_subsystem(tailSubsystem)
    heatMap.set_deltaZ(3)
    heatMap.set_fileOut(fileOut)
    print("HeatMapMatrix")
    heatMap.HeatMapMatrix()
    print("heatMapPlot")

    heatMap.heatMapPlot()


    
    return subsystem
    


def main():    
    subsystemList=["MNWN"]
    list(map(getAllPlots,subsystemList))


if __name__ == "__main__":
    main()

HeatMapMatrix
0       0.000653
1       0.000611
2       0.000660
3       0.000624
4       0.000627
5       0.000627
6       0.000632
7       0.000599
8       0.000647
9       0.000638
10      0.000642
11      0.000613
12      0.000638
13      0.000600
14      0.000650
15      0.000627
16      0.000653
17      0.000624
18      0.000643
19      0.000625
20      0.000654
21      0.000637
22      0.000649
23      0.000625
24      0.000647
25      0.000630
26      0.000673
27      0.000646
28      0.000705
29      0.000643
          ...   
9969    0.000000
9970    0.000000
9971    0.000000
9972    0.000000
9973    0.000000
9974    0.000000
9975    0.000000
9976    0.000000
9977    0.000000
9978    0.000000
9979    0.000000
9980    0.000000
9981    0.000000
9982    0.000000
9983    0.000000
9984    0.000000
9985    0.000000
9986    0.000000
9987    0.000000
9988    0.000000
9989    0.000000
9990    0.000000
9991    0.000000
9992    0.000000
9993    0.000000
9994    0.000000
9995    0.000000


TypeError: unsupported operand type(s) for /: 'float' and 'NoneType'

In [9]:
sum1(2,3,4)

NameError: name 'agrs' is not defined

In [47]:
import sys
import os
sys.path.append('/home/hbhattar/afs/Hemanta/metals/pythonScripts/')
print(sys.path)

['/home/hbhattar/anaconda3/lib/python36.zip', '/home/hbhattar/anaconda3/lib/python3.6', '/home/hbhattar/anaconda3/lib/python3.6/lib-dynload', '', '/home/hbhattar/.local/lib/python3.6/site-packages', '/home/hbhattar/anaconda3/lib/python3.6/site-packages', '/home/hbhattar/anaconda3/lib/python3.6/site-packages/PyDispatcher-2.0.5-py3.6.egg', '/home/hbhattar/anaconda3/lib/python3.6/site-packages/palettable-3.1.1-py3.6.egg', '/home/hbhattar/anaconda3/lib/python3.6/site-packages/tabulate-0.8.2-py3.6.egg', '/home/hbhattar/anaconda3/lib/python3.6/site-packages/IPython/extensions', '/home/hbhattar/.ipython', 0, '/home/hbhattar/afs/Hemanta/metals/pythonScripts/']


In [45]:
os.system("pwd")

0

In [46]:
from subprocess import call
call("pwd")

0